In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import string
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
rslt = input("Enter a trip advisor url")

In [3]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Chrome/119.0.6045.105 "})
    soup = BeautifulSoup(result.text, 'html.parser')
    return soup

In [ ]:
result = []

html = getAndParseURL(rslt)

try:
    div_elements = html.find_all("div", {"class": "entry"})
    time.sleep(1)
    # print(div_elements)

    reviews = [div.find("p", class_="partial_entry").text for div in div_elements]
    time.sleep(1)
    # print(reviews)
    
except:
    print("Except block is called!")
    reviews = [np.nan]

result.extend([reviews])

In [ ]:
try:
    div_elements = html.find_all("div", {"class": "ui_column is-9"})
    time.sleep(1)
    # print(div_elements)

    ratings = [str(div.find('span', {'class': 'ui_bubble_rating'}).get('class', [''])[-1][7]) for div in div_elements if div.find('span', {'class': 'ui_bubble_rating'}) is not None]
    time.sleep(1)
    # print(ratings)
    
except:
    print("Except block is called!")
    ratings = [np.nan]

result.extend([ratings])

result = [list(item) for item in zip(*result)]

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tag_dict = {'J': wordnet.ADJ,'N': wordnet.NOUN,'V': wordnet.VERB,'R': wordnet.ADV}

In [ ]:
def preprocess_text(text):
    text = str(text).lower()
    clean_text = re.sub('https?://\S+|www\.\S+', '', text) #URLs
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), '', clean_text) #Punctuations
    clean_text = re.sub('\n', '', clean_text) #Backslash n
    clean_text = [word for word in clean_text.split(' ') if word not in stop_words] #Remove stopwords
    clean_text=" ".join(clean_text)
    tokens = clean_text.split()
    pos_tags = nltk.pos_tag(tokens)
    wordnet_tags = [(token, tag_dict.get(tag[0].upper(), wordnet.NOUN)) for token, tag in pos_tags]
    tokens = [lemmatizer.lemmatize(token, tag) for token, tag in wordnet_tags] #Lemmatize
    return " ".join(tokens)